<a href="https://colab.research.google.com/github/ancestor9/2025_Spring_Data-Management/blob/main/week_10/02_Data_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **데이터 모델링**
## **특정 정보 시스템의 요구 사항을 충족하도록 데이터베이스를 설계하고 구조화하는 과정**

<img src ='https://res.cloudinary.com/talend/image/upload/w_1408/q_auto/qlik/glossary/data-modeling/seo-hero-data-modeling_wnhzai.jpg'>


### **3 형식의 문자구조 (S + V + O)**
- **S, O : Customer, Product**
- **V : Order**

# **데이터 정규화 (Data Normalization)**

- **데이터베이스에서 중복을 최소화하고, 데이터의 일관성, 무결성, 효율성을 높이기 위해 테이블을 논리적으로 분해하는 과정**
- **데이터를 작고 의미 있는 단위로 나누어, 관계형 데이터베이스의 이상현상(anomalies)을 방지하는 것이 목적**

## 비정규화된 상태
- **테이블**: `sales_unnormalized`
- **필드**: `sale_id`, `customer_name`, `product_name`, `price`
- **문제점**:
 - 고객 이름이 중복 저장됨
 - 제품 정보(이름, 가격)가 중복 저장됨
 - 데이터 갱신 시 여러 레코드를 수정해야 함

## 제2정규화 후 (제1정규화는 이미 만족)

### Customers 테이블 (3개 필드)
- `customer_id` (PK)
- `name`
- `email`

### Products 테이블 (3개 필드)
- `product_id` (PK)
- `name`
- `price`

### Sales 테이블 (3개 필드)
- `sale_id` (PK)
- `customer_id` (FK)
- `product_id` (FK)

## 정규화의 이점
1. **데이터 중복 제거**: 고객과 제품 정보가 한 번만 저장됨
2. **데이터 일관성**: 가격 변경 시 Products 테이블만 수정
3. **저장 공간 효율성**: 중복 데이터 제거로 공간 절약

## 주의사항
- 데이터 조회 시 JOIN 연산 필요
- 쿼리가 다소 복잡해짐
- 성능을 위해 적절한 인덱스 필요

## **1. 기본 설정 및 모델 정의**

In [1]:
import sqlite3
from datetime import datetime
from pydantic import BaseModel
import pandas as pd

# Pydantic 모델 정의 (각 3개 필드)
class Customer(BaseModel):
    customer_id: int
    name: str
    email: str

class Product(BaseModel):
    product_id: int
    name: str
    price: float

class Sale(BaseModel):
    sale_id: int
    customer_id: int
    product_id: int

# 데이터베이스 설정
def create_connection():
    """SQLite 데이터베이스 연결 생성"""
    conn = sqlite3.connect(':memory:')
    return conn

def print_table_structure(cursor, table_name):
    """테이블 구조 출력"""
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()
    print(f"\n{table_name} 테이블 구조:")
    for col in columns:
        print(f"  {col[1]} ({col[2]})")


## **[Pydantic](https://docs.pydantic.dev/latest/#why-use-pydantic)**


In [2]:
from pydantic import PositiveInt

class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: datetime | None
    tastes: dict[str, PositiveInt]

external_data = {
    'id': 123,
    'signup_ts': '2019-06-01 12:22',
    'tastes': {
        'wine': 9,
        b'cheese': 7,
        'cabbage': '1',
    },
}

user = User(**external_data)

print(user.id)

print(user.model_dump())

123
{'id': 123, 'name': 'John Doe', 'signup_ts': datetime.datetime(2019, 6, 1, 12, 22), 'tastes': {'wine': 9, 'cheese': 7, 'cabbage': 1}}


## **[sqlite3](https://docs.python.org/3/library/sqlite3.html)**

<img src ='https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg'>

In [3]:
# urllib을 사용한 다운로드
import urllib.request
import os

# 기존 파일 삭제
if os.path.exists('Chinook_Sqlite.sqlite'):
    os.remove('Chinook_Sqlite.sqlite')

# 다운로드
url = 'https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite'
urllib.request.urlretrieve(url, 'Chinook_Sqlite.sqlite')

print(f"파일 크기: {os.path.getsize('Chinook_Sqlite.sqlite')} bytes")

파일 크기: 1067008 bytes


In [4]:
# Basic Connection and Exploration
import sqlite3

# Connect to the database
conn = sqlite3.connect('Chinook_Sqlite.sqlite')

# Create a cursor object
cursor = conn.cursor()

# Get all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in Chinook database:")
for table in tables:
    print(f"- {table[0]}")


Tables in Chinook database:
- Album
- Artist
- Customer
- Employee
- Genre
- Invoice
- InvoiceLine
- MediaType
- Playlist
- PlaylistTrack
- Track


In [5]:
# 3. 특정 테이블의 구조 확인
print("\n=== albums 테이블 구조 ===")
cursor.execute("PRAGMA table_info(Album);")
columns = cursor.fetchall()
for column in columns:
    print(f"Column: {column[1]}, Type: {column[2]}")


=== albums 테이블 구조 ===
Column: AlbumId, Type: INTEGER
Column: Title, Type: NVARCHAR(160)
Column: ArtistId, Type: INTEGER


In [6]:
# 4. 간단한 SELECT 쿼리
print("\n=== 처음 5개의 앨범 ===")
cursor.execute("SELECT * FROM Album LIMIT 5;")
albums = cursor.fetchall()
for album in albums:
    print(f"ID: {album[0]}, Title: {album[1]}, ArtistID: {album[2]}")


=== 처음 5개의 앨범 ===
ID: 1, Title: For Those About To Rock We Salute You, ArtistID: 1
ID: 2, Title: Balls to the Wall, ArtistID: 2
ID: 3, Title: Restless and Wild, ArtistID: 2
ID: 4, Title: Let There Be Rock, ArtistID: 1
ID: 5, Title: Big Ones, ArtistID: 3


In [7]:
# 5. JOIN 쿼리 - 아티스트와 앨범 정보 조합
print("\n=== 아티스트별 앨범 (처음 10개) ===")
query = """
SELECT Artist.Name, Album.Title
FROM Artist
JOIN Album ON Artist.ArtistId = Album.ArtistId
LIMIT 10;
"""
cursor.execute(query)
results = cursor.fetchall()
for artist, album in results:
    print(f"Artist: {artist}, Album: {album}")



=== 아티스트별 앨범 (처음 10개) ===
Artist: AC/DC, Album: For Those About To Rock We Salute You
Artist: Accept, Album: Balls to the Wall
Artist: Accept, Album: Restless and Wild
Artist: AC/DC, Album: Let There Be Rock
Artist: Aerosmith, Album: Big Ones
Artist: Alanis Morissette, Album: Jagged Little Pill
Artist: Alice In Chains, Album: Facelift
Artist: Antônio Carlos Jobim, Album: Warner 25 Anos
Artist: Apocalyptica, Album: Plays Metallica By Four Cellos
Artist: Audioslave, Album: Audioslave


In [8]:
# 6. 집계 함수 사용
print("\n=== 장르별 트랙 수 ===")
query = """
SELECT Genre.Name, COUNT(Track.TrackId) as TrackCount
FROM Genre
JOIN Track ON Genre.GenreId = Track.GenreId
GROUP BY Genre.Name
ORDER BY TrackCount DESC
LIMIT 5;
"""
cursor.execute(query)
genre_counts = cursor.fetchall()
for genre, count in genre_counts:
    print(f"Genre: {genre}, Tracks: {count}")


=== 장르별 트랙 수 ===
Genre: Rock, Tracks: 1297
Genre: Latin, Tracks: 579
Genre: Metal, Tracks: 374
Genre: Alternative & Punk, Tracks: 332
Genre: Jazz, Tracks: 130


In [9]:
# 7. Pandas와 함께 사용하기
print("\n=== Pandas로 데이터 읽기 ===")
df = pd.read_sql_query("SELECT * FROM Customer LIMIT 5", conn)
df


=== Pandas로 데이터 읽기 ===


,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [10]:

# 8. 고객별 총 구매액 계산
print("\n=== 고객별 총 구매액 (상위 5명) ===")
query = """
SELECT c.FirstName, c.LastName, SUM(i.Total) as TotalSpent
FROM Customer c
JOIN Invoice i ON c.CustomerId = i.CustomerId
GROUP BY c.CustomerId
ORDER BY TotalSpent DESC
LIMIT 5;
"""
top_customers = pd.read_sql_query(query, conn)
print(top_customers)



=== 고객별 총 구매액 (상위 5명) ===
  FirstName    LastName  TotalSpent
0    Helena        Holý       49.62
1   Richard  Cunningham       47.62
2      Luis       Rojas       46.62
3  Ladislav      Kovács       45.62
4      Hugh    O'Reilly       45.62


In [11]:
# 연결 종료
conn.close()

In [12]:
# prompt: tables를 데이터프레임으로 모두 변경하여 저장하려면


# 데이터베이스 연결
conn = sqlite3.connect('Chinook_Sqlite.sqlite')
cursor = conn.cursor()

# 모든 테이블 이름 가져오기
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# 각 테이블을 데이터프레임으로 변환하여 딕셔너리에 저장
dfs = {}
for table_name in tables:
    table_name = table_name[0]  # 튜플에서 문자열 추출
    query = f"SELECT * FROM {table_name}"
    dfs[table_name] = pd.read_sql_query(query, conn)

# 연결 종료
conn.close()

# 데이터프레임 출력 또는 저장 (예시: CSV 파일로 저장)
for table_name, df in dfs.items():
    print(f"--- {table_name} ---")
    print(df.head())  # 첫 5행 출력

--- Album ---
   AlbumId                                  Title  ArtistId
0        1  For Those About To Rock We Salute You         1
1        2                      Balls to the Wall         2
2        3                      Restless and Wild         2
3        4                      Let There Be Rock         1
4        5                               Big Ones         3
--- Artist ---
   ArtistId               Name
0         1              AC/DC
1         2             Accept
2         3          Aerosmith
3         4  Alanis Morissette
4         5    Alice In Chains
--- Customer ---
   CustomerId  FirstName     LastName  \
0           1       Luís    Gonçalves   
1           2     Leonie       Köhler   
2           3   François     Tremblay   
3           4      Bjørn       Hansen   
4           5  František  Wichterlová   

                                            Company  \
0  Embraer - Empresa Brasileira de Aeronáutica S.A.   
1                                              None

In [ ]:
dfs

{'Album':      AlbumId                                              Title  ArtistId
 0          1              For Those About To Rock We Salute You         1
 1          2                                  Balls to the Wall         2
 2          3                                  Restless and Wild         2
 3          4                                  Let There Be Rock         1
 4          5                                           Big Ones         3
 ..       ...                                                ...       ...
 342      343                             Respighi:Pines of Rome       226
 343      344  Schubert: The Late String Quartets & String Qu...       272
 344      345                                Monteverdi: L'Orfeo       273
 345      346                              Mozart: Chamber Music       274
 346      347  Koyaanisqatsi (Soundtrack from the Motion Pict...       275
 
 [347 rows x 3 columns],
 'Artist':      ArtistId                                        

In [13]:
dfs.keys()

dict_keys(['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track'])

In [14]:
dfs['Album']

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3
...,...,...,...
342,343,Respighi:Pines of Rome,226
343,344,Schubert: The Late String Quartets & String Qu...,272
344,345,Monteverdi: L'Orfeo,273
345,346,Mozart: Chamber Music,274


## **2. 비정규화된 테이블**

In [15]:
# 비정규화된 테이블
def create_unnormalized_table(conn):
    """비정규화된 단일 테이블 생성 (단순화)"""
    cursor = conn.cursor()

    # 모든 정보가 하나의 테이블에 있는 비정규화된 구조
    cursor.execute('''
    CREATE TABLE sales_unnormalized (
        sale_id INTEGER PRIMARY KEY,
        customer_name TEXT,
        product_name TEXT,
        price REAL
    )
    ''')

    # 샘플 데이터 삽입
    sales_data = [
        (1, '홍길동', '노트북', 1500000),
        (2, '홍길동', '마우스', 30000),
        (3, '김철수', '노트북', 1500000),
        (4, '김철수', '키보드', 50000)
    ]

    cursor.executemany('''
    INSERT INTO sales_unnormalized VALUES (?,?,?,?)
    ''', sales_data)

    conn.commit()
    print("비정규화된 테이블 생성 완료")
    print_table_structure(cursor, 'sales_unnormalized')

    # 데이터 조회
    df = pd.read_sql_query("SELECT * FROM sales_unnormalized", conn)
    print("\n비정규화된 데이터:")
    print(df)

    return cursor

In [16]:
conn = create_connection()

create_unnormalized_table(conn)

비정규화된 테이블 생성 완료

sales_unnormalized 테이블 구조:
  sale_id (INTEGER)
  customer_name (TEXT)
  product_name (TEXT)
  price (REAL)

비정규화된 데이터:
   sale_id customer_name product_name      price
0        1           홍길동          노트북  1500000.0
1        2           홍길동          마우스    30000.0
2        3           김철수          노트북  1500000.0
3        4           김철수          키보드    50000.0


## **3. 정규화된 테이블**

In [17]:
# 제1정규화 (1NF)
def first_normal_form(conn):
    """제1정규화: 각 컬럼이 원자값을 가지도록 변경"""
    cursor = conn.cursor()

    print("\n=== 제1정규화 (1NF) ===")
    print("규칙: 각 컬럼은 원자값만 포함해야 함")
    print("현재 테이블은 이미 1NF를 만족합니다.")
    print("- 모든 컬럼이 단일 값을 포함")
    print("- 중복 그룹이 없음")

    return cursor

# 제2정규화 (2NF)
def second_normal_form(conn):
    """제2정규화: 부분 함수 종속성 제거"""
    cursor = conn.cursor()

    print("\n=== 제2정규화 (2NF) ===")
    print("규칙: 기본키에 완전 함수 종속")
    print("분리된 테이블:")

    # Customers 테이블 (3개 필드)
    cursor.execute('''
    CREATE TABLE customers (
        customer_id INTEGER PRIMARY KEY,
        name TEXT,
        email TEXT
    )
    ''')

    # Products 테이블 (3개 필드)
    cursor.execute('''
    CREATE TABLE products (
        product_id INTEGER PRIMARY KEY,
        name TEXT,
        price REAL
    )
    ''')

    # Sales 테이블 (3개 필드)
    cursor.execute('''
    CREATE TABLE sales (
        sale_id INTEGER PRIMARY KEY,
        customer_id INTEGER,
        product_id INTEGER,
        FOREIGN KEY (customer_id) REFERENCES customers (customer_id),
        FOREIGN KEY (product_id) REFERENCES products (product_id)
    )
    ''')

    # 데이터 이전
    # 고객 데이터 이전 (중복 제거)
    cursor.execute('''
    INSERT INTO customers (customer_id, name, email)
    VALUES
        (1, '홍길동', 'hong@email.com'),
        (2, '김철수', 'kim@email.com')
    ''')

    # 제품 데이터 이전 (중복 제거)
    cursor.execute('''
    INSERT INTO products (product_id, name, price)
    VALUES
        (1, '노트북', 1500000),
        (2, '마우스', 30000),
        (3, '키보드', 50000)
    ''')

    # 판매 데이터 이전
    cursor.execute('''
    INSERT INTO sales (sale_id, customer_id, product_id)
    VALUES
        (1, 1, 1),
        (2, 1, 2),
        (3, 2, 1),
        (4, 2, 3)
    ''')

    conn.commit()

    print_table_structure(cursor, 'customers')
    print_table_structure(cursor, 'products')
    print_table_structure(cursor, 'sales')

    return cursor

In [18]:
# conn = create_connection()
first_normal_form(conn)


=== 제1정규화 (1NF) ===
규칙: 각 컬럼은 원자값만 포함해야 함
현재 테이블은 이미 1NF를 만족합니다.
- 모든 컬럼이 단일 값을 포함
- 중복 그룹이 없음


In [19]:
second_normal_form(conn)


=== 제2정규화 (2NF) ===
규칙: 기본키에 완전 함수 종속
분리된 테이블:

customers 테이블 구조:
  customer_id (INTEGER)
  name (TEXT)
  email (TEXT)

products 테이블 구조:
  product_id (INTEGER)
  name (TEXT)
  price (REAL)

sales 테이블 구조:
  sale_id (INTEGER)
  customer_id (INTEGER)
  product_id (INTEGER)


이제 각 테이블이 3개의 필드만 가지도록 단순화

- Customers: customer_id, name, email
- Products: product_id, name, price
- Sales: sale_id, customer_id, product_id

이 단순화된 버전에서는:

- 비정규화된 테이블에는 4개 필드만 있습니다
- 정규화 후 각 테이블은 정확히 3개 필드를 가집니다
- 제3정규화는 생략했습니다 (이미 충분히 정규화됨)
- 데이터 구조를 이해하기 쉽게 만들었습니다

## **4. 전체 실행**

In [20]:
def demonstrate_normalization():
    """정규화 과정 데모"""
    conn = create_connection()

    # 비정규화된 테이블 생성
    create_unnormalized_table(conn)

    # 제1정규화
    first_normal_form(conn)

    # 제2정규화
    second_normal_form(conn)

    # 최종 결과 확인
    print("\n=== 최종 정규화된 데이터 ===")

    # 정규화된 데이터 조인하여 조회
    query = '''
    SELECT
        s.sale_id,
        c.name as customer_name,
        p.name as product_name,
        p.price
    FROM sales s
    JOIN customers c ON s.customer_id = c.customer_id
    JOIN products p ON s.product_id = p.product_id
    '''

    df = pd.read_sql_query(query, conn)
    print("\n정규화된 데이터 (조인 결과):")
    print(df)

    # 각 테이블의 데이터 개별 확인
    print("\n=== 개별 테이블 데이터 ===")
    print("\nCustomers 테이블:")
    print(pd.read_sql_query("SELECT * FROM customers", conn))

    print("\nProducts 테이블:")
    print(pd.read_sql_query("SELECT * FROM products", conn))

    print("\nSales 테이블:")
    print(pd.read_sql_query("SELECT * FROM sales", conn))

    conn.close()

if __name__ == "__main__":
    demonstrate_normalization()

비정규화된 테이블 생성 완료

sales_unnormalized 테이블 구조:
  sale_id (INTEGER)
  customer_name (TEXT)
  product_name (TEXT)
  price (REAL)

비정규화된 데이터:
   sale_id customer_name product_name      price
0        1           홍길동          노트북  1500000.0
1        2           홍길동          마우스    30000.0
2        3           김철수          노트북  1500000.0
3        4           김철수          키보드    50000.0

=== 제1정규화 (1NF) ===
규칙: 각 컬럼은 원자값만 포함해야 함
현재 테이블은 이미 1NF를 만족합니다.
- 모든 컬럼이 단일 값을 포함
- 중복 그룹이 없음

=== 제2정규화 (2NF) ===
규칙: 기본키에 완전 함수 종속
분리된 테이블:

customers 테이블 구조:
  customer_id (INTEGER)
  name (TEXT)
  email (TEXT)

products 테이블 구조:
  product_id (INTEGER)
  name (TEXT)
  price (REAL)

sales 테이블 구조:
  sale_id (INTEGER)
  customer_id (INTEGER)
  product_id (INTEGER)

=== 최종 정규화된 데이터 ===

정규화된 데이터 (조인 결과):
   sale_id customer_name product_name      price
0        1           홍길동          노트북  1500000.0
1        2           홍길동          마우스    30000.0
2        3           김철수          노트북  1500000.0
3        4     